In [5]:
import sqlite3

# Welcome message for the hotel reservation system.
print("************************** Welcome to hotel reservation system **************************")

class room:
    roomList = []  # List to store room objects.

    def __init__(self, roomNo, numberOfPeople, availability):
        # Initialize a room object.
        self.roomNo = roomNo
        self.numberOfpeople = numberOfPeople
        self.availability = availability
        self.roomList.append(self)

    @classmethod
    def addRoom(cls, roomNo, numberOfPeople, availability):
        # Add a room to the room list.
        room(roomNo, numberOfPeople, availability)

    @classmethod
    def printTheRooms(cls):
        # Print the details of all rooms in the room list.
        for room in cls.roomList:
            print(f"Room number: {room.roomNo}, Number of people: {room.numberOfpeople}, Availability of the room: {room.availability}")

class databaseConnection:
    def __init__(self, databaseName, tableName):
        # Initialize a database connection.
        self.databse = sqlite3.connect(databaseName)
        self.cursor = self.databse.cursor()
        self.table = tableName
        # Create the table if it doesn't exist.
        self.cursor.execute(f'CREATE TABLE IF NOT EXISTS {self.table} (ROOM_NO INTEGER PRIMARY KEY AUTOINCREMENT,  NUMBER_OF_PEOPLE INTEGER NOT NULL, ROOM_AVIABILITY INTEGER)')

    def fetchTheData(self):
        # Fetch data from the database and update the room list.
        self.cursor.execute(f"SELECT * FROM {self.table}")
        result = self.cursor.fetchall()
        room.roomList.clear()
        for data in result:
            room.addRoom(data[0], data[1], data[2])

    def addRoomToDatabase(self, numberOfPeople):
        # Add a room with the specified number of people to the database.
        query1 = f"INSERT INTO {self.table} (NUMBER_OF_PEOPLE, ROOM_AVIABILITY) VALUES (?, ?)"
        self.cursor.execute(query1, (numberOfPeople, 1))
        self.databse.commit()
        self.fetchTheData()
        print("Room added successfully.")

    def reservation(self, roomNo):
        # Reserve a room by updating its availability status in the database.
        query2 = f"SELECT ROOM_AVIABILITY FROM {self.table} WHERE ROOM_NO = {roomNo}"
        self.cursor.execute(query2)
        isAvailable = self.cursor.fetchone()[0]
        if isAvailable == 1:
            query3 = f'UPDATE {self.table} SET ROOM_AVIABILITY = 0 WHERE ROOM_NO = {roomNo}'
            self.cursor.execute(query3)
            self.databse.commit()
            self.fetchTheData()
            print("The reservation is complete.")
        else:
            print("Room is already reserved.")

    def search(self, numberOfPeople):
        # Search for available rooms that can accommodate a specified number of people.
        query4 = f'SELECT ROOM_NO FROM {self.table} WHERE NUMBER_OF_PEOPLE = {numberOfPeople}'
        self.cursor.execute(query4)
        self.databse.commit()
        result4 = self.cursor.fetchall()
        print(f"Rooms for {numberOfPeople} people are: ", end=" ")
        for rm in result4:
            print(f"{rm[0]}", end=" ")
        print("")

    def releaseRoom(self, roomNo):
        # Release a reserved room by updating its availability status in the database.
        query3 = f"SELECT ROOM_AVIABILITY FROM {self.table} WHERE ROOM_NO = {roomNo}"
        self.cursor.execute(query3)
        isAvailable = self.cursor.fetchone()[0]
        if isAvailable == 0:
            query3 = f'UPDATE {self.table} SET ROOM_AVIABILITY = 1 WHERE ROOM_NO = {roomNo}'
            self.cursor.execute(query3)
            self.databse.commit()
            self.fetchTheData()
            print("The room is now released.")
        else:
            print("Room is not reserved.")


************************** Welcome to hotel reservation system **************************


In [6]:
database = databaseConnection("reservation_system.db", "room")
database.addRoomToDatabase(5)
database.search(3)
database.reservation(24)
database.releaseRoom(24)

Room added successfully.
Rooms for 3 people are:  23 24 
The reservation is complete.
The room is now released.
